In [1]:
import os
import sys
import numpy as np

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
src_dir = os.path.join(project_dir, 'src')
log_dir = os.path.join(project_dir, 'log')
fig_dir = os.path.join(project_dir, 'fig')

# Add the src directory to sys.path
sys.path.append(src_dir)


from Dataset.cancer_dataset_for_LS import main_generate_cancer_matrices_for_LS, load_cancer_dataset_matrices_for_LS
from analysis.commons import data_normalize_by_features, concatenate_B_b

from estimator.NDIS import BasicNDISEstimator
from analysis.RP_privacy_analysis_advanced import gaussian_projection_ndis_delta_1d

In [2]:
file_X_path = os.path.join(project_dir, "Dataset", "cancer-LR-X.txt")
file_y_path = os.path.join(project_dir, "Dataset", "cancer-LR-y.txt")

X = None
y = None
try:
    X, y = load_cancer_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
except:
    main_generate_cancer_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
    X, y = load_cancer_dataset_matrices_for_LS()

B, b = data_normalize_by_features(X, y)
A = concatenate_B_b(B, b)

In [3]:
x = B[:,1]
r = 300
eps = 0
index_sets = [186, 2, 0]

gaussian_projection_ndis_delta_1d(x, index_sets, r, eps)


0.010641855866782735

In [4]:
mean_1 = np.zeros(r)
mean_2 = np.zeros(r)
normD2 = float(np.sum(x ** 2))

x_prime = x.copy()
x_prime[index_sets] = 0.0
normDp2 = float(np.sum(x_prime ** 2))
cov_1 = np.eye(r)*normD2
cov_2 = np.eye(r)*normDp2

estimator = BasicNDISEstimator(mean_1, cov_1, mean_2, cov_2, workers=50, num_samples=2**16)

method = "pqmc"
print(f"delta of ALS for index {index_sets}: {estimator.estimate(eps, method=method)} with epsilon = {eps}")

delta of ALS for index [186, 2, 0]: {'delta': 0.010627517954170673, 'se': 6.416194775732624e-06} with epsilon = 0


In [5]:
lambda_1 = 2
lambda_2 = 2
reg_delta = gaussian_projection_ndis_delta_1d(x, index_sets, r, eps, lambda_1, lambda_2)

print(f"reg_delta: {reg_delta} with lambda_1 = {lambda_1} and lambda_2 = {lambda_2}")

reg_delta: 0.0021265772448925253 with lambda_1 = 2 and lambda_2 = 2


In [6]:
x_reg = np.append(x, lambda_1)
mean_1 = np.zeros(r)
mean_2 = np.zeros(r)
normD2 = float(np.sum(x_reg ** 2))

x_prime_reg = x_reg.copy()
x_prime_reg[index_sets] = 0.0
x_prime_reg[len(x_prime_reg)-1] = lambda_2
normDp2 = float(np.sum(x_prime_reg ** 2))
cov_1 = np.eye(r)*normD2
cov_2 = np.eye(r)*normDp2

estimator = BasicNDISEstimator(mean_1, cov_1, mean_2, cov_2, workers=50, num_samples=2**16)

method = "pqmc"
print(f"delta of ALS for index {index_sets}: {estimator.estimate(eps, method=method)} with epsilon = {eps}")

delta of ALS for index [186, 2, 0]: {'delta': 0.0021263170819476795, 'se': 9.78900589382732e-07} with epsilon = 0
